In [25]:
import numpy as np
import pandas as pd
import numpy_financial as npf

def main():
    # Project Parameters
    initial_investment = 140_147_295 / 1_000_000  # Initial investment in million £
    annual_revenue = 31_031_324 / 1_000_000       # Annual revenue in million £
    annual_fixed_costs = 7_682_051 / 1_000_000    # Annual fixed costs in million £
    annual_utility_costs = 19_987_224 / 1_000_000 # Annual utility costs in million £
    tax_rate = 0.20                               # Corporate tax rate (20%)
    discount_rate = 0.10                          # Discount rate (10%)
    project_life = 10                             # Project duration in years

    # Salvage Value (Assumed 0 since asset is fully depreciated in 10 years)
    salvage_value = 0

    # Year 0 cashflow (Initial capital outflow)
    cash_flows = [-initial_investment]

    # Lists for tabular storage of financial values over 10 years
    years = list(range(project_life + 1))  # List from 0 to 10
    opening_TWDV_list = []  # Opening Tax Written Down Value (TWDV)
    depreciation_list = []   # Annual depreciation
    ebit_list = []           # Earnings Before Interest & Tax (EBIT)
    taxable_income_list = [] # Taxable income (assumed equal to EBIT)
    tax_list = []            # Tax payments (20% of taxable income)
    net_cf_list = []         # Net Cash Flow
    cumulative_cf_list = []  # Cumulative Cash Flow

    # Initial values for Year 0
    opening_TWDV = float(initial_investment)  # Full investment cost
    cumulative_cf = -initial_investment       # Cumulative cash flow starts negative

    # Append Year 0 values (No depreciation, no EBIT, only investment outflow)
    opening_TWDV_list.append(None)   # Not applicable for Year 0
    depreciation_list.append(0.0)
    ebit_list.append(None)
    taxable_income_list.append(None)
    tax_list.append(None)
    net_cf_list.append(-initial_investment)
    cumulative_cf_list.append(cumulative_cf)

    # Loop through Years 1 to 10 to calculate financial metrics
    for year in range(1, project_life + 1):
        # Store Opening TWDV (beginning of the year value)
        opening_TWDV_list.append(opening_TWDV)

        # Calculate annual depreciation (10% of previous year's TWDV)
        annual_depreciation = opening_TWDV * 0.10
        depreciation_list.append(annual_depreciation)

        # EBIT Calculation (Revenue - Fixed Costs - Utility Costs)
        ebit = annual_revenue - (annual_fixed_costs + annual_utility_costs)
        ebit_list.append(ebit)

        # Taxable Income (EBIT is fully taxable)
        taxable_income = ebit
        taxable_income_list.append(taxable_income)

        # Tax Calculation (20% of taxable income)
        tax = taxable_income * tax_rate
        tax_list.append(tax)

        # Net Cash Flow Calculation (EBIT - Tax)
        net_cf = ebit - tax

        # If last year, add salvage value to net cash flow
        if year == project_life:
            net_cf += salvage_value

        # Store Net Cash Flow
        net_cf_list.append(net_cf)

        # Update Cumulative Cash Flow
        cumulative_cf += net_cf
        cumulative_cf_list.append(cumulative_cf)

        # Update Opening TWDV for next year (TWDV reduces by depreciation amount)
        opening_TWDV -= annual_depreciation

        # Append net cash flow to list for later NPV/IRR calculations
        cash_flows.append(net_cf)

    # Create a Pandas DataFrame to store and display results in tabular form
    data = {
        'Year': years,
        'Opening TWDV': opening_TWDV_list,
        'Depreciation': depreciation_list,
        'EBIT': ebit_list,
        'Taxable Income': taxable_income_list,
        'Tax (@ 20%)': tax_list,
        'Net CF': net_cf_list,
        'Cumulative CF': cumulative_cf_list
    }
    df = pd.DataFrame(data)

    # NPV Calculation (Net Present Value) at a 10% discount rate
    npv_value = sum(cf / ((1 + discount_rate) ** i) for i, cf in enumerate(cash_flows))

    # IRR Calculation (Internal Rate of Return)
    irr_value = npf.irr(cash_flows)  # IRR as a decimal fraction
    irr_percent = irr_value * 100    # Convert to percentage

    # Convert DataFrame to LaTeX table format for reports
    latex_table = df.to_latex(index=False, float_format="%.3f")

    # Print results in console
    print("\n--- 10-Year Cash Flow Table (MM GBP)---")
    print(df.to_string(index=False, float_format='{:,.3g}'.format))
    print(f"\nNPV (at {discount_rate*100:.3g}% discount rate): {npv_value:,.2f} MM GBP")
    print(f"IRR: {irr_percent:,.3g}%")

    # Print LaTeX table format
    print("\n--- LaTeX Table Output ---")
    print(latex_table)

# Ensures the script runs only if executed directly
if __name__ == "__main__":
    main()



--- 10-Year Cash Flow Table (MM GBP)---
 Year  Opening TWDV  Depreciation  EBIT  Taxable Income  Tax (@ 20%)  Net CF  Cumulative CF
    0           NaN             0   NaN             NaN          NaN    -140           -140
    1           140            14  3.36            3.36        0.672    2.69           -137
    2           126          12.6  3.36            3.36        0.672    2.69           -135
    3           114          11.4  3.36            3.36        0.672    2.69           -132
    4           102          10.2  3.36            3.36        0.672    2.69           -129
    5            92           9.2  3.36            3.36        0.672    2.69           -127
    6          82.8          8.28  3.36            3.36        0.672    2.69           -124
    7          74.5          7.45  3.36            3.36        0.672    2.69           -121
    8            67           6.7  3.36            3.36        0.672    2.69           -119
    9          60.3          6.03  3.36